In [3]:
import zipfile
import os
import pandas as pd
import numpy as np
import re
import torch
import torch.nn as nn
from transformers import AutoTokenizer
from models import Transformer

In [29]:
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [30]:
train = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del train['lic']
train.tail()

,src,tar
232731,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
232732,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
232733,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...
232734,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...
232735,I went drinking with one of my boyfriend's fri...,« Je suis allée boire avec un ami de mon compa...


In [31]:
train.tail(1)['src'].values

array(['I went drinking with one of my boyfriend\'s friends, and now he\'s furious at me. "Was this friend a guy or a girl?" "A guy, obviously. Why would I go drinking with his female friends?" "Yeah, you\'re right." "His name is Tom. He\'s really hot, and I really want to go drinking with him again."'],
      dtype=object)

In [32]:
# 정규표현식 활용하여 데이터 전처리
def clean_text(inputString):
    text_rmv = re.sub(r'[\\\xa0·«»]', '', inputString)
    # 다수 개의 공백을 하나의 공백으로 치환
    text_rmv = re.sub(r"\s+", " ", text_rmv)
    return text_rmv

In [33]:
train['tar'] = train['tar'].apply(lambda x: clean_text(x))
train['src'] = train['src'].apply(lambda x: clean_text(x))
train

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !
...,...,...
232731,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
232732,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
232733,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...
232734,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...


In [34]:
total_len = len(train)
total_len

232736

In [35]:
idx = np.random.permutation(total_len)
idx

array([ 16623,  32885,  57639, ..., 222852,  60405, 114345])

In [36]:
int(total_len*0.8)

186188

In [37]:
train_df, test_df = train.iloc[idx[:int(total_len*0.9)]], train.iloc[idx[int(total_len*0.9):]]

In [38]:
train_df.reset_index(inplace=True, drop=True)
train_df

,src,tar
0,It's artificial.,C'est artificiel.
1,He came here again.,Il est revenu ici.
2,I love reading in bed.,J'adore lire au lit.
3,She is always neat and tidy.,Elle est toujours soignée et ordonnée.
4,You're on the wrong road.,Vous êtes sur la mauvaise route.
...,...,...
209457,His achievements were acknowledged.,Ses réalisations ont été saluées.
209458,What do you advise?,Que conseillez-vous ?
209459,The teacher patted me on the back.,Le professeur m'a congratulé.
209460,May I use some of this paper?,Est-ce que je peux utiliser de ce papier?


In [39]:
test_df.reset_index(inplace=True, drop=True)
test_df

,src,tar
0,Why did I get a C?,Pourquoi ai-je obtenu un satisfaisant?
1,I flunked out of school.,J'ai été virée de l'école.
2,I don't want to do that.,Je ne veux pas faire ça.
3,I knew it was Tom.,Je savais que c'était Tom.
4,I do like the way you think.,J'apprécie vraiment votre façon de penser.
...,...,...
23269,There is no solution.,Il n'y a aucune solution.
23270,You can't do this alone.,Vous ne pouvez pas faire ça tout seul.
23271,Education is one of the most essential aspects...,Il n'y a rien de plus important dans la vie qu...
23272,That's not your knife.,Ce n'est pas votre couteau.


In [40]:
train.tail(1)['src'].values[0]

'I went drinking with one of my boyfriend\'s friends, and now he\'s furious at me. "Was this friend a guy or a girl?" "A guy, obviously. Why would I go drinking with his female friends?" "Yeah, you\'re right." "His name is Tom. He\'s really hot, and I really want to go drinking with him again."'

In [41]:
train.tail(1)['tar'].values[0]

"Je suis allée boire avec un ami de mon compagnon, et voilà qu'il est furieux contre moi. Était-ce un gars ou une fille? Un gars, bien évidemment. Pourquoi irais-je boire avec ses amies? Ouais, ça se comprend. Il s'appelle Tom. Il est trop canon, et j'ai tellement envie d'aller prendre un verre avec lui à nouveau."

In [42]:
checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [43]:
print('영어 최대 길이: ', np.max(train['src'].apply(lambda x: len(tokenizer.tokenize(x)))))
print('불어 최대 길이: ', np.max(train['tar'].apply(lambda x: len(tokenizer.tokenize(x)))))

영어 최대 길이:  88
불어 최대 길이:  112


In [44]:
tokenizer.add_special_tokens({'bos_token': '<s>'})
print(tokenizer.bos_token)
print(tokenizer.eos_token)
print(tokenizer.pad_token)

<s>
</s>
<pad>


In [45]:
tokenizer.vocab_size

32100

In [46]:
tokenizer.decode(0)

'<pad>'

In [47]:
print(tokenizer.pad_token_id)

0


In [48]:
max_len = 120

In [49]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.src = self.tokenizer(list(self.data['src']), padding=True, truncation=True, max_length = self.max_len, return_tensors='pt').input_ids
        self.tar = self.tokenizer(['<s>' + s for s in self.data['tar']], padding=True, truncation=True, max_length = self.max_len, return_tensors='pt').input_ids
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        return self.src[idx], self.tar[idx]

In [50]:
custom_ds = CustomDataset(train, tokenizer, max_len)
train_ds, valid_ds, test_ds = torch.utils.data.random_split(custom_ds, [0.8, 0.1, 0.1])
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=64, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=64, shuffle=True)

In [51]:
len(train_dl)

2910

In [52]:
i = 6
idx = train_ds.indices[i]
print(idx)
src_text, trg_text = custom_ds.__getitem__(idx)
print(src_text)
print(tokenizer.decode(src_text))
print(trg_text)
print(tokenizer.decode(trg_text))

2031
tensor([   27, 30312,    25,     5,     1,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0])
I envy you.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [53]:
i = 6
idx = valid_ds.indices[i]
print(idx)
src_text, trg_text = custom_ds.__getitem__(idx)
print(src_text)
print(tokenizer.decode(src_text))
print(trg_text)
print(tokenizer.decode(trg_text))

79723
tensor([1615,   33,   79, 4973,   53,   58,    1,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0])
Why are they protesting?</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [54]:
i = 6
idx = test_ds.indices[i]
print(idx)
src_text, trg_text = custom_ds.__getitem__(idx)
print(src_text)
print(tokenizer.decode(src_text))
print(trg_text)
print(tokenizer.decode(trg_text))

74822
tensor([  27,  217, 2971,   16,   39, 2053,    5,    1,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0])
I see fear in your eyes.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [55]:
src_texts, tar_texts = next(iter(train_dl))
print(src_texts.shape)
print(src_texts[0])
print(tokenizer.decode(src_texts[0]))
print(tar_texts.shape)
print(tar_texts[0])
print(tokenizer.decode(tar_texts[0]))

torch.Size([64, 89])
tensor([ 290,   47,    3,    9,  508, 2417,   44, 4981,   31,    7, 4219,    5,
           1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0])
There was a large audience at yesterday's concert.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [11]:
vocab_size = tokenizer.vocab_size + 1
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 120
dropout = 0.1
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

model = Transformer(vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout, device).to(device)

In [57]:
print(model)

Transformer(
  (encoder_embedding): Embedding(32101, 512)
  (decoder_embedding): Embedding(32101, 512)
  (positional_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (self_attn): MultiHeadAttentionLayer(
        (W_q): Linear(in_features=512, out_features=512, bias=False)
        (W_k): Linear(in_features=512, out_features=512, bias=False)
        (W_v): Linear(in_features=512, out_features=512, bias=False)
        (W_o): Linear(in_features=512, out_features=512, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (feed_forward): PositionwiseFeedForward(
        (fc_1): Linear(in_features=512, out_features=2048, bias=True)
        (fc_2): Linear(in_features=2048, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1,

In [58]:
src_texts = src_texts.to(device)
tar_texts = tar_texts.to(device)
output, attention = model(src_texts, tar_texts[:, :-1])

In [59]:
src_texts.shape

torch.Size([64, 89])

In [60]:
tar_texts.shape

torch.Size([64, 114])

In [61]:
output.shape

torch.Size([64, 113, 32101])

In [62]:
attention.shape

torch.Size([64, 8, 113, 89])

In [5]:
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
tokenizer.add_special_tokens({'bos_token':'<s>'})

1

In [7]:
src_tensor = tokenizer(['That was pathetic'], padding=True, truncation=True, max_length = 120, return_tensors='pt').input_ids[0].unsqueeze(0)

In [10]:
src_tensor.unsqueeze(0).shape

torch.Size([1, 1, 5])

In [9]:
src_tensor.shape

torch.Size([1, 5])

In [13]:
src_tensor.shape

torch.Size([1, 5])

In [14]:
model.make_src_mask(src_tensor)

tensor([[[[True, True, True, True, True]]]])

In [15]:
src_mask = model.make_src_mask(src_tensor)
enc_output = src_tensor
with torch.no_grad():
    for enc_layer in model.encoder_layers:
        enc_output = enc_layer(enc_output, src_mask)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_CUDA_mm)